<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Drought Model

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook demos drought modelling.

In [123]:
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load, fit, utils

In [170]:
import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [172]:
setup.plotting()

In [ ]:
%load_ext autoreload
%autoreload 2

Load the raw FIA grouped data

In [174]:
df = load.fia(store='local', states='conus', group_repeats=True)

Compute mortality ratios

In [186]:
df['mort_ratio'] = df['mort_1'] / df['balive_0']

Set thresholds

In [184]:
inds = (
    (df['condprop'] > 0.3) & 
    (df['disturb_human_1']!=True) & 
    (df['disturb_fire_1']!=True) & 
    (df['treatment_cutting_1']!=True) & 
    (df['balive_0'] > 0) &
    (~np.isnan(df['mort_1']))
)

Make a map of mortality ratios

In [187]:
plot.carto(
    lat=df2['lat'], 
    lon=df2['lon'], 
    color=df2['mort_ratio'],
    cmap='reds',
    clim=(0,0.3),
    size=2,
)

alt.LayerChart(...)

Make a downsampled map

In [188]:
final_mask = load.nlcd(store='local', year=2016, classes='all')

In [189]:
gridded = fit.interp(df[inds], final_mask, var='mort_ratio').coarsen(x=8, y=8, boundary='trim').mean().values

In [190]:
r, c = np.nonzero(gridded)
lat, lon = utils.rowcol_to_latlon(r, c, res=32000)

In [185]:
len(df[inds])

100684

In [191]:
plot.carto(
    lat=lat, 
    lon=lon, 
    color=gridded[r, c],
    cmap='reds',
    clim=(0,0.3),
    size=15,
    clabel='mort_ratio'
)

alt.LayerChart(...)